In [ ]:
!pip install ffmpeg-python
!pip install "tensorflow-gpu<2" "dm-sonnet<2" "tensorflow-probability==0.7.0"
!git clone https://github.com/deepmind/kinetics-i3d
!cp /content/kinetics-i3d/i3d.py i3d.py

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import cv2
import shutil
import json
import numpy as np
import ffmpeg  
import math
import pickle
from scipy.special import softmax
import tensorflow as tf
import os
import time
import i3d
import random
def center_standardization(img):
  img=img.astype('float')
  mean=np.mean(img)
  std=np.std(img)
  for x in range(len(img)):
    for y in range(len(img[0])):
      for z in range(len(img[0,0])):
        img[x,y,z]=(img[x,y,z]-mean)/max(std,1/img.size)
  return img
def generate_i3d_feature(video_path):
  video_path,width,height=change_fps(video_path,3)
  cap = cv2.VideoCapture(video_path)
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  i3d_features=np.zeros((frame_count,224,224,3))
  i3d_features_path='i3d_features'
  i=0
  while 1:
    ret,frame=cap.read()
    if not ret:
      break
    diff=abs(width-height)
    if width>height:
      frame=cv2.resize(frame[:,diff//2:-diff//2],(224,224))
    elif height>width:
      frame=cv2.resize(frame[diff//2:-diff//2],(224,224))
    else:
      frame=cv2.resize(frame,(224,224))
    i3d_features[i]=center_standardization(frame)
    i+=1
  i3d_features_path='i3d_features'
  if os.path.exists(i3d_features_path):
    !rm -rf 'i3d_features'
  os.makedirs(i3d_features_path)
  for i in range(0,frame_count-8,3):
    np.save(i3d_features_path+'/'+'{}.npy'.format(i//3),i3d_features[i:i+9])
  return i3d_features_path
def predict_i3d(i3d_features_path):
  tf.reset_default_graph()
  _BATCH_SIZE = 32
  _CLIP_SIZE = 9
  _FRAME_SIZE = 224
  _CHECKPOINT_PATHS = {
      'rgb':'/content/_RGB_0.754_model-34986'
  }
  clip_holder=tf.placeholder(tf.float32,shape=(_BATCH_SIZE, _CLIP_SIZE, _FRAME_SIZE, _FRAME_SIZE, 3))
  with tf.variable_scope('RGB'):
    model = i3d.InceptionI3d()
    logits, _ = model(clip_holder, is_training=False,dropout_keep_prob=1)
    logits_dropout = tf.nn.dropout(logits, 1)
    fc_out = tf.layers.dense(logits_dropout, 49, use_bias=True)
  variable_map = {}
  for variable in tf.global_variables():
    tmp = variable.name.split('/')
    if tmp[0] == 'RGB':
      variable_map[variable.name.replace(':0', '')] = variable
  saver = tf.train.Saver(var_list=variable_map)
  sess = tf.Session()
  sess.run(tf.global_variables_initializer())
  saver.restore(sess, _CHECKPOINT_PATHS['rgb'])
  num_step = int(np.ceil(len(os.listdir(i3d_features_path))/_BATCH_SIZE))
  gen=data_generator(i3d_features_path,_BATCH_SIZE)
  true_count = 0
  dic={}
  for i in range(num_step):
    x_t,names=next(gen)
    fc_out2= sess.run(fc_out,feed_dict={clip_holder:x_t})
    preds=softmax(fc_out2,axis=1)
    for j in range(len(names)):
      name=names[j]
      pred=preds[j]
      dic[int(name)]=(np.argmax(pred),max(pred))
  # to ensure every test procedure has the same test size
  sess.close()
  times=list(dic.keys())
  times.sort()
  preds=[]
  probs=[]
  for i in range(len(times)):
    pred,prob=dic[times[i]]
    preds.append(pred)
    probs.append(prob)
  return preds,probs,times
def data_generator(path,batch_size):
    dirs=[]
    for file in os.listdir(path):
        dirs.append(path+'/'+file)
    random.shuffle(dirs)
    i=0
    while True:
        if i+batch_size<=len(dirs):
            batch_dirs=dirs[i:i+batch_size]
            i+=batch_size
        else:
            batch_dirs=dirs[i:]
            random.shuffle(dirs)
            batch_dirs+=dirs[:i+batch_size-len(dirs)]
            i=i+batch_size-len(dirs)
        dim=np.load(batch_dirs[0]).shape[-1]
        batch_x=np.zeros((batch_size,9,224,224,dim))
        names=[]
        for j in range(len(batch_dirs)):
            input_dir=batch_dirs[j]
            batch_x[j]=np.load(input_dir)
            name=input_dir.split('/')[-1][:-4]
            names.append(name)
        yield(batch_x,names)
def check_rotation(path_video_file):
    # this returns meta-data of the video file in form of a dictionary
    meta_dict = ffmpeg.probe(path_video_file)
    # from the dictionary, meta_dict['streams'][0]['tags']['rotate'] is the key
    # we are looking for
    rotateCode = None
    if 'tags' in meta_dict['streams'][0] and 'rotate' in meta_dict['streams'][0]['tags']:
      if int(meta_dict['streams'][0]['tags']['rotate']) == 90:
          rotateCode = cv2.ROTATE_90_CLOCKWISE
      elif int(meta_dict['streams'][0]['tags']['rotate']) == 180:
          rotateCode = cv2.ROTATE_180
      elif int(meta_dict['streams'][0]['tags']['rotate']) == 270:
          rotateCode = cv2.ROTATE_90_COUNTERCLOCKWISE
    return rotateCode
def change_fps(dir,out_fps):
  rotateCode = check_rotation(dir)
  cap = cv2.VideoCapture(dir)
  fps = cap.get(cv2.CAP_PROP_FPS)
  width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  if rotateCode:
    if rotateCode!=cv2.ROTATE_180:
      tmp=width
      width=height
      height=temp
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  duration = frame_count/fps
  out_frames=int(duration*out_fps)
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  dir2=dir[:-4]+'_changed.avi'
  out = cv2.VideoWriter(dir2,fourcc, out_fps, (width,height))
  cnt=0
  ret=True
  i=0
  while cnt<frame_count and ret:
    if cnt/fps>=1800:
      break
    ret,frame=cap.read()
    if ret:
      if cnt==i*frame_count//out_frames:
        i+=1
        if rotateCode is not None:
            frame = cv2.rotate(frame, rotateCode)
        out.write(frame)
    cnt+=1
  out.release()
  cap.release()
  return dir2,width,height
def predict_rgb(video,videoId):
  with open('/content/label_map.txt', 'rb') as f:
    label_map = pickle.load(f)
  i3d_features_path=generate_i3d_feature(video)
  if len(os.listdir(i3d_features_path))==0:
    return 'Video too short',None
  preds,probs,times=predict_i3d(i3d_features_path)
  output=[]
  for i in range(len(times)):
    output.append((label_map[preds[i]],probs[i],times[i]))
  moments=[]
  for i in output:
    if i[0]=='chest pain':
      if moments:
        if moments[-1][0][1]<i[2]:
          moments.append([[i[2],i[2]+3],i[1]])
        else:
          moments[-1][0][1]=i[2]+3
          moments[-1][1]=max(moments[-1][1],i[1])

      else:
        moments.append([[i[2],i[2]+3],i[1]])
  print(moments)
  data=[]
  for moment in moments:
    [start,end],prob=moment
    data.append({'videoId':videoId,'type':'segment','startTime':start,'endTime':end,'observer':'luyipeng','isHuman':False,'confirmedBySomeone':False,'rejectedBySomeone':False,'observation':{'label':'chest pain','labelConfidence':str(prob)}})
  with open('resultOfDetection', 'w') as outfile:
    json.dump(data, outfile)
  return output,moments
videoPath='/content/test_youtube.mp4'
videoId='le5JRilSMvc'
output,moments=predict_rgb(videoPath,videoId)